In [1]:
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *
from pandas import merge, read_excel, pivot_table, melt, to_datetime, to_timedelta, DateOffset, Timestamp, Period, Series
import numpy as np
import pandas as pd
import re
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month)

def diff_days(start_date, end_date):
    # 날짜 간 차이 계산
    delta = end_date - start_date
    # 차이를 일자로 변환하여 반환
    return delta.days


In [2]:
origin = read_excel("통합9-2.xlsx")
origin['양도'].fillna(0, inplace=True)
origin['환불여부'].fillna(0, inplace=True)
origin['환불내역'].fillna("", inplace=True)
origin['요일'].fillna("", inplace=True)
origin['추세값'].fillna(0, inplace=True) # 1개 행 
origin['광고'].fillna(0, inplace=True) # 1개 행 
origin= origin.drop(['Unnamed: 0'], axis=1)
트레이너 = ['알수없음', '전진용', '서하은', '홍석민', '황규선', '김광현', '홍훈기', '김영찬',
       '이상현', '백희정', '고형범', '임우혁', '김강', '문건주', '홍석남', '박근열', '금소현',
       '조병창', '오수정', '이수환', '이상준', '김강민', '강대현', '김소현']
for i in range(len(origin)):
    if origin.loc[i, '트레이너']  not in 트레이너:
        origin.loc[i, '트레이너'] = ""
origin['트레이너 성별'].fillna(0, inplace=True)
origin['연월'] = to_datetime(origin['연월'])
origin['종료시점'] = to_datetime(origin['종료시점'])

df = origin.copy()
#origin['등록명'].unique()

In [3]:
df1 = df.drop(['최초등록월','휴면가능','휴면반영','마지막거래'],axis=1)
df1

,회원카드번호,연월,등록명,개월수,종료시점,환불여부,양도,거래순서,잔여개월수,결제시점,...,환불사용금매출,의류매출,분실매출,GX매출,상환액매출,입회비매출,예치금매출,FC 성별,트레이너,트레이너 성별
0,12131325,2020-09-01,헬스,12,2021-09-01,0.0,0.0,0,12,0,...,0,0,0,0,0,0,0,2,,0.0
1,13110008,2021-05-01,골프,12,2022-06-01,0.0,0.0,0,13,0,...,0,0,0,0,0,0,0,1,,0.0
2,13110008,2022-06-01,골프,12,2023-08-01,0.0,0.0,1,14,0,...,0,220000,0,0,0,0,0,1,,0.0
3,13110008,2023-11-01,골프,12,2024-11-01,0.0,0.0,2,12,-3,...,0,0,0,0,0,0,0,1,,0.0
4,13110009,2020-10-01,헬스,12,2021-12-01,0.0,0.0,0,14,0,...,0,0,0,0,0,0,0,0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,23120075,2023-12-01,골프,10,2024-10-01,0.0,0.0,0,10,0,...,0,30000,0,0,0,0,0,0,,0.0
6305,23120076,2023-12-01,헬스,10,2024-10-01,0.0,0.0,0,10,0,...,0,0,0,0,0,0,0,0,,0.0
6306,23120077,2023-12-01,헬스,10,2024-10-01,0.0,0.0,0,10,0,...,0,0,0,0,0,0,0,0,,0.0
6307,24010001,2023-12-01,헬스,10,2024-10-01,0.0,0.0,0,10,0,...,0,0,0,0,0,0,0,0,,0.0


In [4]:
df1['GX매출'].unique()

array([     0, 480000,  60000, 470000,  30000, 440000, 888000,  50000,
       150000], dtype=int64)

In [5]:
df1['거래순서'] += 1
df1['거래횟수'] = df1['거래순서']
df2 = df1.drop('거래순서',axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6309 entries, 0 to 6308
Data columns (total 43 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   6309 non-null   int64         
 1   연월       6309 non-null   datetime64[ns]
 2   등록명      6309 non-null   object        
 3   개월수      6309 non-null   int64         
 4   종료시점     6309 non-null   datetime64[ns]
 5   환불여부     6309 non-null   float64       
 6   양도       6309 non-null   float64       
 7   잔여개월수    6309 non-null   int64         
 8   결제시점     6309 non-null   int64         
 9   이탈여부     6309 non-null   int64         
 10  광고       6309 non-null   float64       
 11  추세값      6309 non-null   float64       
 12  환불내역     6309 non-null   object        
 13  요일       6309 non-null   object        
 14  미수액      6309 non-null   int64         
 15  거래일_일    6309 non-null   int64         
 16  환불일_일    6309 non-null   int64         
 17  일일입장     6309 non-null   int64   

In [6]:
# a = test2.query("회원카드번호 ==13110019").sort_values(by='연월')
# a.reset_index(drop=True, inplace=True)
# a

# if (a.iloc[0, 2] != 0) & (a.iloc[0, 3] != 0) & (a.iloc[0,2] <= a.iloc[0, 3]):
#     a.at[a.index[0], '환불시일'] = a.iloc[0, 3] - a.iloc[0, 2]
    
# a

NameError: name 'a' is not defined

In [7]:
test = df2.filter(['회원카드번호','연월', '거래일_일', '환불일_일'])
test2 = test[(test['거래일_일']!=0)|(test['환불일_일']!=0)]
test2['환불시일'] = 0

회원카드 = test2['회원카드번호'].unique()
행병합 = []

for i in 회원카드:
    a = test2.query("회원카드번호 ==@i").sort_values(by='연월')
    a.reset_index(drop=True, inplace=True)
    
    if (a.iloc[0, 2] != 0) & (a.iloc[0, 3] != 0) & (a.iloc[0,2] <= a.iloc[0, 3]):
        a.at[a.index[0], '환불시일'] = a.iloc[0, 3] - a.iloc[0, 2]
    
    for j in range(1, len(a)):
        if (a.iloc[j, 2] != 0) & (a.iloc[j, 3] != 0) & (a.iloc[j,2] <= a.iloc[j, 3]):
            a.at[a.index[j], '환불시일'] = a.iloc[j,3] - a.iloc[j,2]
        elif (a.iloc[j-1,2] != 0) & (a.iloc[j, 3] != 0): # 다른 연월에 환불이 일어난 경우
            a.at[a.index[j], '환불시일'] = diff_days(a.iloc[j-1,1], a.iloc[j,1]) + (a.iloc[j, 3] - a.iloc[j-1,2])
    
    행병합.append(a)

환불시일 = pd.concat(행병합, ignore_index=True)

In [54]:
환불시일

,회원카드번호,연월,거래일_일,환불일_일,환불시일
0,12131325,2020-09-01,8,0,0
1,13110008,2021-05-01,26,0,0
2,13110008,2022-06-01,29,0,0
3,13110008,2023-11-01,6,0,0
4,13110009,2020-10-01,30,0,0
...,...,...,...,...,...
6306,23120075,2023-12-01,29,0,0
6307,23120076,2023-12-01,29,0,0
6308,23120077,2023-12-01,29,0,0
6309,24010001,2023-12-01,31,0,0


In [8]:
df3 =merge(df2, 환불시일, on=['회원카드번호', '연월','거래일_일','환불일_일'], how='left')

In [9]:
df3[(df3['환불시일']!=0)&(df3['골프전환']==1)]

,회원카드번호,연월,등록명,개월수,종료시점,환불여부,양도,잔여개월수,결제시점,이탈여부,...,분실매출,GX매출,상환액매출,입회비매출,예치금매출,FC 성별,트레이너,트레이너 성별,거래횟수,환불시일
924,15110040,2020-10-01,골프,0,2020-10-01,0.0,0.0,0,0,1,...,0,0,0,0,0,1,,0.0,3,93
2018,19030052,2021-06-01,골프,0,2021-08-01,0.0,0.0,2,2,1,...,0,0,0,0,0,1,,0.0,3,144
4539,21110041,2022-06-01,골프,0,2022-06-01,0.0,0.0,0,-1,1,...,0,0,0,0,0,1,,0.0,2,202
4592,21120002,2022-07-01,골프,0,2023-02-01,0.0,0.0,7,7,1,...,0,0,0,0,0,1,,0.0,3,114
5258,22070048,2022-09-01,골프골프골프헬스,1,2022-10-01,0.0,0.0,1,-1,1,...,0,0,0,0,0,2,,0.0,2,64
5540,22120022,2023-09-01,골프,0,2024-08-01,0.0,0.0,11,11,2,...,0,0,0,0,0,2,,0.0,3,96


In [57]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6311 entries, 0 to 6310
Data columns (total 44 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   회원카드번호   6311 non-null   int64         
 1   연월       6311 non-null   datetime64[ns]
 2   등록명      6311 non-null   object        
 3   개월수      6311 non-null   int64         
 4   종료시점     6311 non-null   datetime64[ns]
 5   환불여부     6311 non-null   float64       
 6   양도       6311 non-null   float64       
 7   잔여개월수    6311 non-null   int64         
 8   결제시점     6311 non-null   int64         
 9   이탈여부     6311 non-null   int64         
 10  광고       6311 non-null   float64       
 11  추세값      6311 non-null   float64       
 12  환불내역     6311 non-null   object        
 13  요일       6311 non-null   object        
 14  미수액      6311 non-null   int64         
 15  거래일_일    6311 non-null   int64         
 16  환불일_일    6311 non-null   int64         
 17  일일입장     6311 non-null   int64   

In [10]:
df3.to_excel('통합10-1.xlsx',index=False)